In [1]:
from datetime import datetime
import traceback
import functools
import multiprocessing
from multiprocessing import Pool

import time
import os
import sys
from pathlib import Path

import numpy as np
import scipy
import pandas as pd
import ipympl
import matplotlib
#matplotlib.use('nbagg')
from matplotlib import style
style.use('ggplot')
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

from astropy import stats

import pandas as pd

import astropy.units as u
from astropy.io import fits
from astropy.coordinates import Angle
from mmtwfs.wfs import WFSFactory

In [2]:
# instantiate all of the WFS systems...
wfs_keys = ['f9', 'newf9', 'f5', 'mmirs', 'binospec']
wfs_systems = {}
wfs_names = {}
for w in wfs_keys:
    wfs_systems[w] = WFSFactory(wfs=w)
    wfs_names[w] = wfs_systems[w].name
plt.close('all')

# give mmirs a default
wfs_systems['mmirs'].default_mode = 'mmirs1'

# map f9 to oldf9
wfs_systems['oldf9'] = wfs_systems['f9']

In [3]:
def hist_compare(df1, df2, column, bins=100, mini=None, maxi=None):
    f, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
    if mini is None:
        mini = df1[column].min()
    if maxi is None:
        maxi = df1[column].max()
    df1.hist(column=column, bins=bins, range=(mini, maxi), alpha=0.6, ax=ax1)
    df2.hist(column=column, bins=bins, range=(mini, maxi), alpha=0.6, ax=ax2)
    plt.xlabel(column)
    plt.ylabel("N")
    f.show()

In [19]:
def seeing(df, wfs):
    """
    Given a sigma derived from a gaussian fit to a WFS spot, deconvolve the systematic width from the reference image
    and relate the remainder to r_0 and thus a seeing FWHM.
    """
    # the effective wavelength of the WFS imagers is about 600-650 nm. we use 650 nm to maintain consistency
    # with the value used by the old SHWFS system.
    wave = 550 * u.nm
    wave = wave.to(u.m).value  # r_0 equation expects meters so convert
    owave = 500 * u.nm
    owave = owave.to(u.m).value
    
    mode = wfs.default_mode
    
    # calculate the physical size of each aperture.
    ref = wfs.modes[mode]['reference']
    apsize_pix = np.max((ref.xspacing, ref.yspacing))
    d = wfs.telescope.diameter * apsize_pix / wfs.pup_size
    d = d.to(u.m).value  # r_0 equation expects meters so convert

    # we need to deconvolve the instrumental spot width from the measured one to get the portion of the width that
    # is due to spot motion
    ref_sigma = stats.funcs.gaussian_fwhm_to_sigma * ref.fwhm
    sigma = stats.funcs.gaussian_fwhm_to_sigma * df['gauss_fwhm']
    
    corr_sigma = np.sqrt(sigma**2 - ref_sigma**2)
    corr_sigma *= wfs.pix_size.to(u.rad).value  # r_0 equation expects radians so convert

    # this equation relates the motion within a single aperture to the characteristic scale size of the
    # turbulence, r_0.
    r_0 = ( 0.179 * (wave**2) * (d**(-1/3))/corr_sigma**2 )**0.6

    # this equation relates the turbulence scale size to an expected image FWHM at the given wavelength.
    raw_seeing = u.Quantity(u.rad * 0.98 * owave / r_0, u.arcsec)

    # correct seeing to zenith
    seeing = raw_seeing / df['AIRMASS']**0.6

    return raw_seeing, seeing

In [20]:
columns = ['filename', 'obstime', 'wfskey', 'AIRMASS', 'AZ', 'EL', 'ellipticity', 'gauss_rms', 'moffat_rms', 'gauss_fwhm', 'moffat_fwhm', 'gauss_amplitude',
           'moffat_amplitude', 'moffat_alpha', 'moffat_gamma']
rootdir = Path("/Volumes/Seagate2TB/spot_analysis")
dirs = sorted(list(rootdir.glob("20*")))  # pathlib, where have you been all my life!
dataframes = []
for d in dirs:
    if d.is_dir():
        mean_df = pd.read_csv(d / "spot_reduce_mean.csv")
        wfs_df = pd.read_csv(d / "wfskeys.csv")
        df = pd.merge(mean_df, wfs_df, on='filename')
        dataframes.append(df)
data = pd.concat(dataframes, sort=True)
data['moment_fwhm'] = data['moment_fwhm'] * stats.gaussian_fwhm_to_sigma  # this looks like a photuils bug where sigma is actually fwhm and i applied correction i shouldn't have

In [21]:
g = data.groupby('wfskey')
g.wfskey.describe()

,count,unique,top,freq
wfskey,,,,
binospec,16996,1,binospec,16996
f5,100654,1,f5,100654
mmirs,49707,1,mmirs,49707
newf9,1803,1,newf9,1803
oldf9,54128,1,oldf9,54128


In [29]:
# re-calculate the seeing using updated spot FWHM for binospec. filter out any NaN values...
fixed_list = []
for wfskey, group in g:
    wfs = wfs_systems[wfskey]
    group = g.get_group(wfskey)
    #group.apply(seeing, axis=1, args=(wfs,))
    group['raw_seeing'], group['seeing'] = seeing(group, wfs)
    fixed_list.append(group)
fixed = pd.concat(fixed_list)
fixed = fixed[np.isfinite(fixed['seeing'])]
fixed = fixed[fixed['gauss_amplitude'] > 10.]

fixed = fixed[~fixed['filename'].str.contains("Ref")]
fixed = fixed[~fixed['filename'].str.contains("ref")]
fixed = fixed[~fixed['filename'].str.contains("back.fits")]
fixed = fixed[~fixed['filename'].str.contains("tmp.fits")]
fixed = fixed[fixed['seeing'] > 0.2]  # trim out reference or bogus images
fixed = fixed[fixed['seeing'] < 5.]  # spot finding doesn't work for bad seeing where motion fills the aperture

fixed = fixed[fixed['peak'] > 2500.]  # use only bright spots

# create a date-time index so we can group and analyze the data by timestamps
fixed = fixed.set_index(pd.DatetimeIndex(fixed['obstime'], name='ut'))

/Users/tim/conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in sqrt
/Users/tim/conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [30]:
fixed.hist(column='ellipticity', bins=100, range=(0.0, 1.0), alpha=0.6)
plt.title("Ellipticity")
plt.xlabel("e")
plt.ylabel("N")
plt.show()

FigureCanvasNbAgg()

In [31]:
# make raw histogram of the zenith-corrected seeing column
fixed.hist(column='seeing', bins=100, range=(0.0, 5.0), alpha=0.6)
plt.title("March 2003 through May 2018")
plt.xlabel("Seeing (arcsec)")
plt.ylabel("N")
plt.show()

FigureCanvasNbAgg()

In [32]:
len(fixed)

121526

In [33]:
plt.close('all')

In [34]:
# fit a log-normal distribution to the seeing data
sigma, loc, exp_mu = scipy.stats.lognorm.fit(fixed['seeing'])
print(sigma, loc, exp_mu)
x = np.arange(0.0, 4.0, 0.01)
p = scipy.stats.lognorm.pdf(x, sigma, loc=loc, scale=exp_mu)
mu = np.log(exp_mu)
mode = np.exp(mu - sigma**2) + loc
median = np.nanmedian(fixed['seeing'])
fit_median = exp_mu + loc

0.464922345053 0.13788246283 0.735357321241


In [35]:
# plot normalized histogram with the fit and the median/mode calculated from the fit.
plt.hist(fixed['seeing'], density=True, bins=100, range=(0.0, 4.0), label="hist", alpha=0.6)
logp = plt.plot(x, p, label="logp")
plt.xlabel("Seeing (arcsec)")
plt.ylabel("Number Density")
plt.title("March 2003 through May 2018")
plt.legend(["median=%.2f\", mode=%.2f\"" % (fit_median, mode), "median=%.2f\"" % (
    median
)])
plt.savefig("all_seeing.pdf")
plt.show()

FigureCanvasNbAgg()

In [15]:
yearly_median = []
for i in range(2004, 2018):
    year = str(i)
    med = fixed[year]['seeing'].median()
    print(f"Median seeing for {year} is {med}")
    yearly_median.append(med)

Median seeing for 2004 is 0.9147829424423
Median seeing for 2005 is 0.8673004918976338
Median seeing for 2006 is 0.8485604398722123
Median seeing for 2007 is 0.9420881703619992
Median seeing for 2008 is 0.8707950631404169
Median seeing for 2009 is 0.8221425913378688
Median seeing for 2010 is 0.9232630038098257
Median seeing for 2011 is 0.8765445296404224
Median seeing for 2012 is 0.8548792557440318
Median seeing for 2013 is 0.8799431311418863
Median seeing for 2014 is 0.8482458930693173
Median seeing for 2015 is 0.8600815832798809
Median seeing for 2016 is 0.8854704658997801
Median seeing for 2017 is 0.8146719971515832


In [19]:
plt.scatter(range(2004, 2018), yearly_median)
plt.title("Median Seeing by Year")
plt.xlabel("Year")
plt.ylabel("Seeing (arcsec)")
plt.ylim(0.5, 1.5)
plt.savefig("yearly_median.pdf")
plt.show()

FigureCanvasNbAgg()

In [20]:
# now use the between_time() method to split the data into first and second halves of the night.
first_half = fixed.between_time(start_time='00:00', end_time='07:00')
second_half = fixed.between_time(start_time='07:00', end_time='14:00')
#first_half = months['December'].between_time(start_time='00:00', end_time='07:00')
#second_half = months['December'].between_time(start_time='07:00', end_time='14:00')
med_1st = np.median(first_half['seeing'])
med_2nd = np.median(second_half['seeing'])

In [21]:
# plot the results and show that the seeing is better later in the night...
plt.hist(first_half['seeing'], bins=100, range=(0.0, 4.0), label="1st", alpha=0.6)
plt.hist(second_half['seeing'], bins=100, range=(0.0, 4.0), label="2nd", alpha=0.6)
plt.legend(["1st Half: %.2f\"" % med_1st, "2nd Half: %.2f\"" % med_2nd])
plt.xlabel("Seeing (arcsec)")
plt.ylabel("N")
#plt.title("Data and median seeing values grouped by half of the night")
plt.savefig("half_nights.pdf")
plt.show()

FigureCanvasNbAgg()

In [22]:
months = ["{:02d}".format(m) for m in range(1, 13)]
years = ["{:d}".format(y) for y in range(2003, 2019)]
months, years

(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
 ['2003',
  '2004',
  '2005',
  '2006',
  '2007',
  '2008',
  '2009',
  '2010',
  '2011',
  '2012',
  '2013',
  '2014',
  '2015',
  '2016',
  '2017',
  '2018'])

In [23]:
by_month = fixed.groupby([fixed.index.month]).median()

In [24]:
by_month['seeing']

ut
1     0.906379
2     0.955259
3     0.862584
4     0.962720
5     0.864881
6     0.890816
7     0.827072
8     0.763875
9     0.801844
10    0.839930
11    0.815623
12    0.821637
Name: seeing, dtype: float64

In [25]:
month_sizes = [len(g[1]) for g in fixed.groupby([fixed.index.month]).groups.items()]

In [67]:
mnames = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
fig, ax = plt.subplots()
x = range(12)
ax.scatter(x, by_month['seeing'])
ax.set_xticks(x)
ax.set_xticklabels(mnames, rotation=45)
ax.set_ylabel("Median Seeing (arcsec)")
ax.set_ylim(0.5, 1.5)
plt.savefig("monthly_median.pdf")
plt.show()

/Users/tim/conda/envs/py36/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [28]:
fig, ax = plt.subplots()
x = range(12)
plt.bar(x, month_sizes)
ax.set_xticks(x)
ax.set_xticklabels(mnames, rotation=45)
ax.set_ylabel("Samples per Month")
plt.savefig("monthly_samples.pdf")
plt.show()

FigureCanvasNbAgg()

In [29]:
bino1_dir = Path("/Users/tim/MMT/bino/GDR1")
bino2_dir = Path("/Users/tim/MMT/bino/GDR2")
dfs = []
for c in bino1_dir.glob("*.csv.gz"):
    df = pd.read_csv(c)
    dfs.append(df)
bino1 = pd.concat(dfs)

dfs = []
for c in bino2_dir.glob("*.csv.gz"):
    df = pd.read_csv(c)
    dfs.append(df)
bino2 = pd.concat(dfs)

In [30]:
bino1['airmass'] = 1.0 / np.cos((90. - bino1['EL']).data*u.deg)
bino2['airmass'] = 1.0 / np.cos((90. - bino2['EL']).data*u.deg)
bino1['seeing'] = bino1['gauss_fwhm'] / bino1['airmass']**0.6
bino2['seeing'] = bino2['gauss_fwhm'] / bino2['airmass']**0.6
bino1['mseeing'] = bino1['moffat_fwhm'] / bino1['airmass']**0.6
bino2['mseeing'] = bino2['moffat_fwhm'] / bino2['airmass']**0.6

/Users/tim/conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Series.data is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.
/Users/tim/conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Series.data is deprecated and will be removed in a future version
  


In [31]:
#fixed = fixed.set_index(pd.DatetimeIndex(fixed['obstime'], name='ut'))
bino1_t = bino1.set_index(pd.DatetimeIndex(bino1['DATE-OBS'], name='ut'))
bino2_t = bino2.set_index(pd.DatetimeIndex(bino2['DATE-OBS'], name='ut'))

In [32]:
plt.hist(bino1_t['seeing'], bins=100, range=(0.0, 4.0), label="Gaussian", alpha=0.6)
plt.hist(bino1_t['mseeing'], bins=100, range=(0.0, 4.0), label="Moffat", alpha=0.6)
plt.legend(["Gaussian: %.2f\"" % np.nanmedian(bino1_t['seeing']), "Moffat: %.2f\"" % np.nanmedian(bino1_t['mseeing'])])
plt.xlabel("Seeing (arcsec)")
plt.ylabel("N")
#plt.title("Data and median seeing values grouped by half of the night")
plt.savefig("gauss_v_moffat.pdf")
plt.show()

/Users/tim/conda/envs/py36/lib/python3.6/site-packages/numpy/lib/function_base.py:748: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= mn)
/Users/tim/conda/envs/py36/lib/python3.6/site-packages/numpy/lib/function_base.py:749: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= mx)


FigureCanvasNbAgg()

In [33]:
plt.hist(bino1_t['mseeing'], bins=100, range=(0.0, 4.0), label="Camera #1", alpha=0.6)
plt.hist(bino2_t['mseeing'], bins=100, range=(0.0, 4.0), label="Camera #2", alpha=0.6)
plt.legend(["Camera #1: %.2f\"" % np.nanmedian(bino1_t['mseeing']), "Camera #2: %.2f\"" % np.nanmedian(bino2_t['mseeing'])])
#plt.legend()
plt.xlabel("Seeing (arcsec)")
plt.ylabel("N")
#plt.title("Data and median seeing values grouped by half of the night")
plt.savefig("gdr1_v_gdr2.pdf")
plt.show()

/Users/tim/conda/envs/py36/lib/python3.6/site-packages/numpy/lib/function_base.py:748: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= mn)
/Users/tim/conda/envs/py36/lib/python3.6/site-packages/numpy/lib/function_base.py:749: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= mx)


FigureCanvasNbAgg()

In [34]:
bino_wfs = fixed[fixed['wfskey'].str.contains('binospec')]
bino2018 = bino_wfs['2018']

In [36]:
plt.hist(bino2018['seeing'], bins=100, range=(0.0, 4.0), density=True, label="WFS", alpha=0.6)
plt.hist(bino2_t['seeing'], bins=100, range=(0.0, 4.0), density=True, label="Guider", alpha=0.6)
plt.legend(["WFS: %.2f\"" % np.nanmedian(bino2018['seeing']), "Guider: %.2f\"" % np.nanmedian(bino2_t['seeing'])])
#plt.legend()
plt.xlabel("Seeing (arcsec)")
plt.ylabel("Number Density")
#plt.title("Data and median seeing values grouped by half of the night")
plt.savefig("wfs_v_gdr.pdf")
plt.show()

FigureCanvasNbAgg()

In [37]:
bino2018['EXPTIME'].mean(), bino2_t['EXPTIME'].mean()

(38.892494205740775, 1.5120392333396819)

In [39]:
# make a new data frame that resamples the WFS data daily into the median, min, and max seeing for each month.
fig, ax = plt.subplots()
daily = pd.DataFrame()
daily['seeing'] = fixed.seeing.resample('M').median()
daily['max'] = fixed.seeing.resample('M').max()
daily['min'] = fixed.seeing.resample('M').min()
lowb = daily['seeing'] - daily['min']
upb = daily['max'] - daily['seeing']
#plt.errorbar(daily.index, daily['seeing'], yerr=[lowb, upb], fmt='o')
plt.scatter(daily.index, daily['seeing'])
labels = ax.get_xticklabels()
plt.setp(labels, rotation=20, fontsize=8, ha='right')
plt.ylabel("Seeing (arcsec)")
plt.title("Median Seeing by Month")
plt.savefig("monthly_median.pdf")
plt.show()

FigureCanvasNbAgg()

In [40]:
hourly_medians = []
times = []
hourly_sizes = []
for h in range(1, 13):
    stime = "{:02d}:00".format(h)
    mtime = "{:02d}:30".format(h)
    etime = "{:02d}:00".format(h+1)
    tslice = fixed.between_time(start_time=stime, end_time=etime)
    med = np.median(tslice['seeing'])
    hourly_medians.append(med)
    hourly_sizes.append(len(tslice))
    times.append(mtime)

In [41]:
hourly_medians, hourly_sizes

([0.88295897782304011,
  0.9116116830832488,
  0.88039964017880268,
  0.91714198756608578,
  0.88559538180614439,
  0.85404149514943295,
  0.85646935724899143,
  0.85672448039685167,
  0.84823477982966822,
  0.84525693976945426,
  0.82864618882704588,
  0.81592869290404924],
 [7665,
  11487,
  13763,
  11462,
  12739,
  12697,
  11535,
  11129,
  10512,
  9074,
  6191,
  2570])

In [42]:
fig, ax = plt.subplots()
x = range(12)
ax.scatter(x, hourly_medians)
ax.set_xticks(x)
ax.set_xticklabels(times, rotation=45)
ax.set_ylabel("Median Seeing (arcsec)")
ax.set_title("Median Seeing vs. UT")
plt.savefig("hourly_median.pdf")
plt.show()

FigureCanvasNbAgg()

In [43]:
fig, ax = plt.subplots()
x = range(12)
plt.bar(x, hourly_sizes)
ax.set_xticks(x)
ax.set_xticklabels(times, rotation=45)
ax.set_ylabel("Samples per Hour")
plt.savefig("hourly_samples.pdf")
plt.show()

FigureCanvasNbAgg()

In [44]:
len(data), len(fixed)

(223288, 121526)

In [45]:
for c in fixed.columns:
    if 'wdir' in c.lower():
        print(c)
    if 'wind' in c.lower():
        print(c)

WDIR_E
WDIR_W
WIND
WINDDIR
WIND_E
WIND_W


In [46]:
fixed['wind_speed'] = fixed['WIND']
fixed['wind_speed'] = fixed['wind_speed'].fillna(fixed['WIND_W'])
fixed['wind_dir'] = fixed['WINDDIR']
fixed['wind_dir'] = fixed['wind_dir'].fillna(fixed['WDIR_W'])
wfixed = fixed[fixed['wind_speed'] > 0.0]

In [47]:
wfixed['wind_dir'].median()

213.0

In [48]:
wfixed.hist(column='wind_speed', bins=50, range=(0.0, 50.0), alpha=0.6)
plt.title("")
plt.xlabel("Wind Speed (mph)")
plt.ylabel("N")
plt.savefig("wind_hist.pdf")
plt.show()

FigureCanvasNbAgg()

In [49]:
wfixed.hist(column='wind_dir', bins=180, range=(0.0, 360.0), alpha=0.6)
plt.title("")
plt.xlabel("Wind Direction (deg)")
plt.ylabel("N")
plt.savefig("winddir_hist.pdf")
plt.show()

/Users/tim/conda/envs/py36/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [50]:
winds = np.arange(2, 42, 4)
list(winds)

[2, 6, 10, 14, 18, 22, 26, 30, 34, 38]

In [51]:
wmeds = []
for w in winds:
    wl = w - 2
    wu = w + 2
    dslice = wfixed[(wfixed['wind_speed'] >= wl) & (wfixed['wind_speed'] < wu)]
    med = dslice['seeing'].median()
    wmeds.append(med)
wmeds

[0.86291280060177189,
 0.78127101981416369,
 0.78112972824842353,
 0.82227261555283437,
 0.87131455066429764,
 0.97621667146931612,
 0.99992366662732923,
 1.115754877790625,
 1.1235364514850368,
 1.0962254375964573]

In [52]:
fig, ax = plt.subplots()
ax.scatter(winds, wmeds)
ax.set_ylim(0.5, 1.5)
ax.set_xticks(winds)
ax.set_ylabel("Median Seeing (arcsec)")
ax.set_xlabel("Wind Speed (mph)")
plt.savefig("seeing_v_wind.pdf")
plt.show()

/Users/tim/conda/envs/py36/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [53]:
az = np.arange(22.5, 360, 22.5)
az
aznames = ['NNE', 'NE', "ENE", 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW']

In [54]:
ameds = []
for a in az:
    al = a - 22.5
    au = a + 22.5
    dslice = wfixed[(wfixed['wind_dir'] >= al) & (wfixed['wind_dir'] < au)]
    med = dslice['seeing'].median()
    ameds.append(med)
ameds

[0.98410274342373261,
 0.98125217262352471,
 0.97871142393699495,
 0.93284688660782811,
 0.87822829748479192,
 0.8699651708709959,
 0.85659593696825975,
 0.84399175362143608,
 0.85651247507085615,
 0.84782164642830171,
 0.81193275339672477,
 0.81158646399684753,
 0.8636604486785433,
 0.85931807243677893,
 0.89209353155351423]

In [55]:
fig, ax = plt.subplots()
ax.scatter(az, ameds)
ax.set_ylim(0.5, 1.5)
ax.set_xticks(az)
ax.set_xticklabels(aznames, rotation=45, size='x-small')
ax.set_ylabel("Median Seeing (arcsec)")
ax.set_xlabel("Wind Direction")
ax.xaxis.set_label_position('top')
plt.savefig("seeing_v_winddir.pdf")
plt.show()

/Users/tim/conda/envs/py36/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [56]:
tel_wind_offset = wfixed['AZ'] - wfixed['wind_dir']
tel_wind_angle = Angle(tel_wind_offset.data * u.deg)
tel_wind_angle.wrap_at('180d', inplace=True)
wfixed['tel_wind_angle'] = np.abs(tel_wind_angle)

/Users/tim/conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Series.data is deprecated and will be removed in a future version
  
/Users/tim/conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [57]:
wfixed['tel_wind_angle'].median()

92.553891859999908

In [58]:
angles = range(10, 180, 10)
tameds = []
for a in angles:
    al = a - 10
    au = a + 10
    dslice = wfixed[(wfixed['tel_wind_angle'] >= al) & (wfixed['tel_wind_angle'] < au)]
    med = dslice['seeing'].median()
    tameds.append(med)
tameds

[0.8039245932493011,
 0.80047555886688759,
 0.81660233827547057,
 0.84099318413087132,
 0.84372433510238021,
 0.83551916310524232,
 0.82963685400563669,
 0.8193780743569411,
 0.82435782041830397,
 0.86453101892897188,
 0.90930677810162119,
 0.9142893110400947,
 0.92872247207684011,
 0.93793408677524703,
 0.93948166777576558,
 0.94748482880671803,
 0.93148205421833608]

In [59]:
fig, ax = plt.subplots()
ax.scatter(angles, tameds)
ax.set_ylim(0.5, 1.5)
ax.set_xticks(angles)
ax.set_ylabel("Median Seeing (arcsec)")
ax.set_xlabel("Azimuth/Wind Direction Offset (deg)")
ax.xaxis.set_label_position('top')
ax.set_xticklabels(angles, rotation=45, size='small')
plt.savefig("seeing_v_telaz.pdf")
plt.show()

/Users/tim/conda/envs/py36/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [60]:
short = fixed[fixed['EXPTIME'] <= 5.0]
medium = fixed[(fixed['EXPTIME'] > 5.0) & (fixed['EXPTIME'] < 20.0)]
long = fixed[fixed['EXPTIME'] >= 20.0]

fixed.hist(column='EXPTIME', bins=100, range=(0, 60), alpha=0.6)
plt.title("WFS Exposure Times")
plt.ylabel("N")
plt.xlabel("Exposure Time (sec)")
plt.text(60, 31500, "<= 5 sec -> {:0.2f}\"".format(short['seeing'].median()), horizontalalignment='right')
plt.text(60, 28500, "5-20 sec -> {:0.2f}\"".format(medium['seeing'].median()), horizontalalignment='right')
plt.text(60, 25500, ">= 20 sec -> {:0.2f}\"".format(long['seeing'].median()), horizontalalignment='right')
plt.savefig("exptime_v_seeing.pdf")
plt.show()

/Users/tim/conda/envs/py36/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [61]:
short = fixed[fixed['EXPTIME'] <= 5.0]
medium = fixed[(fixed['EXPTIME'] > 5.0) & (fixed['EXPTIME'] < 20.0)]
long = fixed[fixed['EXPTIME'] >= 20.0]
short['seeing'].median(), medium['seeing'].median(), long['seeing'].median()

(0.82026377924795824, 0.93482981669916676, 1.0114399793479858)

In [62]:
for c in fixed.columns:
    if 'exp' in c.lower():
        print(c)

EXPTIME


In [65]:
expfixed = fixed[fixed['EXPTIME'] <= 7.]

# fit a log-normal distribution to the seeing data
sigma, loc, exp_mu = scipy.stats.lognorm.fit(expfixed['seeing'])
print(sigma, loc, exp_mu)
x = np.arange(0.0, 4.0, 0.01)
p = scipy.stats.lognorm.pdf(x, sigma, loc=loc, scale=exp_mu)
mu = np.log(exp_mu)
mode = np.exp(mu - sigma**2) + loc
median = np.nanmedian(expfixed['seeing'])
fit_median = exp_mu + loc

0.452678414843 0.148376889725 0.691541136619


In [66]:
# plot normalized histogram with the fit and the median/mode calculated from the fit.
plt.hist(expfixed['seeing'], density=True, bins=100, range=(0.0, 4.0), label="hist", alpha=0.6)
logp = plt.plot(x, p, label="logp")
plt.xlabel("Seeing (arcsec)")
plt.ylabel("Number Density")
plt.title("Exposure Time < 7 sec")
plt.legend(["median=%.2f\", mode=%.2f\"" % (fit_median, mode), "median=%.2f\"" % (
    median
)])
plt.savefig("short_exp_seeing.pdf")
plt.show()

/Users/tim/conda/envs/py36/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [77]:
pixsize = wfs_systems['binospec'].pix_size.value
reffwhm = wfs_systems['binospec'].modes['binospec']['reference'].fwhm
simple = np.sqrt(bino2018['gauss_fwhm']**2 - reffwhm**2)

In [78]:
plt.hist(simple*pixsize, density=True, bins=100, range=(0.0, 4.0), label="hist", alpha=0.6)
plt.hist(bino2018['seeing'], bins=100, range=(0.0, 4.0), density=True, label="WFS", alpha=0.6)
plt.show()

/Users/tim/conda/envs/py36/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [95]:
plt.hist(bino2018['moment_fwhm']*gaussian_fwhm_to_sigma, bins=100, range=(0.0, 30.0),  alpha=0.6)
plt.hist(bino2018['gauss_fwhm'], bins=100, range=(0.0, 10.0),  alpha=0.6)
plt.hist(bino2018['semiminor_axis_sigma'], bins=100, range=(0.0, 10.0), alpha=0.6)
plt.show()

FigureCanvasNbAgg()

In [88]:
for c in bino2018.columns:
    print(c)

ABS_HUM
ACTUALX
ACTUALY
ADCBUSYE
ADCBUSYW
ADCCPRME
ADCCPRMW
ADCELEV
ADCEPRSE
ADCEPRSW
ADCPARTY
ADCPRES
ADCPRSME
ADCPRSMW
ADCTEMP
ADCTRACK
ADCWAVE1
ADCWAVE2
AEROFF
AIRMASS
AMB_TEMP
AZ
AZOFF
AZ_INST
AZ_OFF
BITPIX
BSCALE
BZERO
CA
CAMERA
CAMTEMP
CATCOORD
CATDEC
CATDEC2K
CATDECPM
CATEPOCH
CATID
CATPA
CATRA
CATRA2K
CATRAPM
CAT_ID
CCD-TEMP
CCDTEMP
CCDXBIN
CCDYBIN
CD1_1
CD1_2
CD2_1
CD2_2
CDELT1
CDELT2
CHAM_DPT
CHAM_RH
CHAM_T
CROTA2
CRPIX1
CRPIX2
CRVAL1
CRVAL2
CURRDEC
CURREPCH
CURRPA
CURRRA
CURRTEMP
DARKTIME
DATAMAX
DATAMIN
DEC
DECOFF
DEC_2000
DEC_OFF
DEWPOINT
EL
ELOFF
EL_INST
EL_OFF
ENDTIME
EPOCH
EQUINOX
EXPTIME
EXTEND
EXTVER
FILTER
FOCUS
FOOTIME
GAIN
GAZOFF
GCOUNT
GELOFF
GRTOFF
GSEEING
GUIDERX
GUIDERY
IAZOFF
IELOFF
INHERIT
INPOS
IROTOFF
IXOFF
IYOFF
JD
LINEARX
LINEARY
LTM1_1
LTM1_2
LTM2_1
LTM2_2
LTV1
LTV2
MASK
MIX_RAT
MJD
MJD-OBS
NAXIS
NAXIS1
NAXIS2
NCOMBINE
OBJECT
OFFSET1
OFFSET2
OSSTEMP
OUT_DPT
OUT_RH
OUT_T
PA
PCOUNT
PIXSIZE1
PIXSIZE2
POSANG
PRESSURE
P_BARO
RA
RAOFF
RA_2000
RA_OFF
RDROFF
REL

In [90]:
from astropy.stats import gaussian_fwhm_to_sigma, gaussian_sigma_to_fwhm

In [92]:
gaussian_sigma_to_fwhm

2.3548200450309493